<a href="https://colab.research.google.com/github/worldbank/dec-python-course/blob/main/3-other-languages/Python-para-ciencia-de-datos/Sesion%205%20-%20Analisis%20de%20textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesion 5 - Analisis de texto descriptivo y clasificacion de textos

Esta ultima sesion cubre los temas de analisis descriptivo de textos y clasificacion de textos.

1. Analisis descriptivo de datos de textos
    1. Conteo de palabras
1. Clasificación de textos

Esta sesión asume los conocimientos impartidos en las 4 sesiones previas: conocimiento basico de Python, conocimiento de pandas, y visualizacion de datos. La sesion parte desde el resultado generado en la sesion 4, que guardamos como un dataframe de pandas en formato pickle.

Usaremos las siguientes bibliotecas en este notebook:

- **seaborn**, **matplotlib** y **wordcloud** para visualización de datos  
- **nltk** para análisis de sentimiento  
- **sklearn** para clasificación de datos

Empezaremos leyendo el dataframe que obtuvimos ayer:

In [ ]:
import pandas as pd

In [ ]:
df_ruta = 'datos/detalle_ventas_clasificadas.pkl'
df = pd.read_pickle(df_ruta)

In [ ]:
df.head()

# 1. Analisis descriptivo de textos

## 1.1 Conteo de palabras

Conteo de palabras consiste en contar:

1. Cuantas palabras tiene un texto o corpus (conjunto de textos)
1. Cuantas veces una palabra se repite en un texto o corpus

Calcularemos ambos resultados en nuestro analisis. Para la primera tarea, crearemos directamente una columna con la longitud de la columna `nombre_item_final` en el dataframe.

**Importante:** aunque es posible aplicar un conteo de palabras sobre textos sin preparacion (*raw*), este no es el mejor metodo para obtener una aproximacion a cuanta informacion contiene un texto ya que los textos sin preparacion incluyen *stop words*, codigos y palabras que no son realmente significativas en nuestro analisis. Por eso, siempre es mejor aplicar el conteo de palabras sobre un texto ya preparado.

### Numero total de palabras en textos

In [ ]:
# Hacemos el conteo con .apply(len)
df['n_palabras'] = df['nombre_item_final'].apply(len)

In [ ]:
df.head()

Veamos ahora la distribucion del numero de palabras en los textos:

In [ ]:
import seaborn as sns

In [ ]:
# Histograma con seaborn
sns.histplot(data=df, x='n_palabras');

In [ ]:
df['n_palabras'].value_counts()

In [ ]:
# Que observacion tendra 8 palabras?
df[df['n_palabras']==8]

In [ ]:
# Observando el string en nombre_item
df[df['n_palabras']==8]['nombre_item'].iloc[0]

### Estimando cuantas veces las palabras se repiten a traves de todos los textos

Para esta tarea, necesitamos generar una función que cree un diccionario donde cada clave es un palabra unica y cada valor un conteo de la palabra, para todos nuestros textos.

In [ ]:
def conteo_de_palabras_unicas(lista_de_palabras):
    
    conteo = {}
    
    for palabra in lista_de_palabras:
        if palabra in conteo:  # esto verifica si la palabra ya existe en las claves de "conteo"
            conteo[palabra] += 1
        else:                  # si la palabra no existe, se anade 1 a su contador
            conteo[palabra] = 1
    
    return conteo

Primero aplicaremos la función a un solo texto para asegurarnos de que el resultado se vea correcto.

In [ ]:
lista_de_palabras = df['nombre_item_final'][42]
lista_de_palabras

In [ ]:
conteo = conteo_de_palabras_unicas(lista_de_palabras)
conteo

El resultado se ve correcto. Pero esto no es muy informativo:

- nuestros textos en cada observacion de `nombre_item_final` son muy pequenos como para que un conteo de palabras a nivel de texto individual nos de informacion util
- probablemente todos los conteos tengan una frecuencia de 1

En lugar de aplicar esta funcion por texto individual, lo aplicaremos a todos los textos.

In [ ]:
# Concatenando todas las listas en nombre_item_final:
nombre_item_total = df['nombre_item_final'].sum()

In [ ]:
nombre_item_total

Por que `.sum()` concatena todas las listas en `nombre_item_final`? `.sum()` es un metodo de pandas que suma todos los valores en una columna. En la sesion 1, sin embargo, vimos que Python permite ejecutar una operacion de **adicion de listas**: el resultado es listas concatenadas. Entonces, al ser aplicado a una columna con listas, `.sum()` resulta en una adicion de todas las listas en la columna, concatenandolas en una sola lista.

Ahora aplicaremos la funcion a la lista con todas las palabras y guardaremos el resultado en `conteo_total`.

In [ ]:
conteo_total = conteo_de_palabras_unicas(nombre_item_total)

In [ ]:
conteo_total

In [ ]:
len(conteo_total)

Con esto, podemos graficar el conteo de palabras para todo nuestro corpus de artículos. Lo haremos a continuación para las *n* palabras más utilizadas.

In [ ]:
n = 15

# Esta linea retorna los valores en conteo_total, en orden descendiente
valores_descendentes = sorted(conteo_total.values())[::-1]

valor_n = valores_descendentes[n]

In [ ]:
valor_n

Esto significa que después de ordenar nuestros conteos de palabras en orden descendente, 8 es el valor en la posición 16 —recuerda que en Python las posiciones siempre empiezan en cero. Vamos a crear un bucle a traves del diccionario y nos quedaremos solo con los conteos mayores a este valor, guardando el resultado en un nuevo diccionario llamado `conteo_total_mayores_n`.

In [ ]:
conteo_total_mayores_n = {}
for palabra, conteo in conteo_total.items():
    if conteo > valor_n:
        conteo_total_mayores_n[palabra] = conteo

In [ ]:
conteo_total_mayores_n

In [ ]:
len(conteo_total_mayores_n)

Ahora podemos producir nuestro plot:

In [ ]:
conteo_total_mayores_n

In [ ]:
conteo_total_mayores_n['crema']

In [ ]:
sorting = sorted(conteo_total_mayores_n, key=lambda x: conteo_total_mayores_n[x]) # anadimos esto para ordenar los resultados

plot = sns.barplot(conteo_total_mayores_n, orient='h', order=sorting[::-1])
plot.set(xticks=list(range(1, 20, 2)));

## 1.2 Nube de palabras

**Nota importante:** los dos paquetes que vamos a usar en el resto de la sesion, `wordcloud` y `scikit-learn`, no se pueden implementar en JupyterLite.

¿Pero qué clase de taller sobre análisis de texto sería este sin un ejemplo de nube de palabras? Usaremos nuestro diccionario de conteos de palabras para el corpus de artículos y la librería `wordcloud` para esto.

In [ ]:
# Activa e instala esta linea para instalar el paquete wordcloud
#!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Wordcloud con las palabras en blanco y negro
wc = WordCloud(background_color='white', colormap = 'binary').generate_from_frequencies(conteo_total)
plt.axis("off")
plt.imshow(wc);

In [ ]:
# Con las palabras a color
wc = WordCloud(background_color='white').generate_from_frequencies(conteo_total)
plt.axis("off")
plt.imshow(wc);

# 2. Clasificacion de textos

Para la última parte de la sesión, haremos un par de ejemplos simples de clasificación de texto. Las llamamos "simples" porque hoy en día existen técnicas muy avanzadas para clasificación de texto, pero no son adecuadas para el tiempo que tenemos en esta sesión. Puedes consultar el enlace que aparece más abajo sobre LLMs si quieres explorar más sobre estos métodos (en ingles).

En términos basicos, la clasificación de texto consiste en asignar un texto a un grupo. Si estás familiarizado con el aprendizaje automático (*machine learning*), clasificacion de textos es una tarea de clasificación. Para nuestro ejercicio, mostraremos dos formas de clasificar texto:
1. **Clasificación supervisada:** agruparemos textos en grupos predefinidos. Los grupos predefinidos serán las clases `Alimentos`, `Bebidas alcohólicas` y `Prendas de vestir y calzado` de la columna `CLASIFICACION`. Todas las demas clases las agruparemos en la clase `Otros`. Hacemos esto porque las demas clases no cuentan con un numero de ejemplos suficientes para producir un clasificador aceptable.
1. **Clasificación no supervisada:** agruparemos textos en grupos según su similitud, sin predefinir los grupos.

**Notas sobre clasificacion supervisada:** 
- Un buen clasificador para clasificacion supervisada normalmente necesita algunos de miles de ejemplos **por clase** para lograr una clasificacion robusta. En este ejemplo, vamos a omitir ese detalle.
- Clasificadores pre-entrenados con una tarea de analisis de texto generica, como los modelos BERT o RoBERTa, son una excepcion a esto y logran buenos resultados con algunos cientos de ejemplos por clase. Puedes leer mas sobre BERT y RoBERTa en el link al final de esta presentacion sobre LLMs.

La variable objectivo (*target*) es la clasificacion. Tendremos cuatro clases:
- Alimentos
- Bebidas alcoholicas
- Prendas de vestir y calzado
- Otros

In [ ]:
# tabulacion de las clasificacion
df['CLASIFICACION'].value_counts()

In [ ]:
# numero de valores en clasificacion
df['CLASIFICACION'].nunique()

Para continuar, crearemos una nueva columna con las clases a predecir para la clasificacion supervisada. La llamaremos `clase`:

In [ ]:
# clase alimentos
df.loc[
    df['CLASIFICACION'] == 'Alimentos',
    'clase'
] = 'alimentos'

In [ ]:
# clase bebidas alcoholicas
df.loc[
    df['CLASIFICACION'] == 'Bebidas alcohólicas',
    'clase'
] = 'bebidas alcoholicas'

In [ ]:
# clase prendas de vestir y clazado
df.loc[
    df['CLASIFICACION'] == 'Prendas de vestir y calzado',
    'clase'
] = 'prendas y calzado'

In [ ]:
# las demas
df.loc[
    df['clase'].isna(),
    'clase'
] = 'otro'

In [ ]:
df['clase'].value_counts()

In [ ]:
df.head()

## 2.1 Codificación de textos (*text encoding*)

Nuestro clasificador será construido (entrenado) usando los textos tokenizados y normalizados en `nombre_item_final`. Sin embargo, primero necesitamos convertirlos en números para que un clasificador pueda trabajar con ellos. Esta operación se llama **codificación** (*encoding*).

Existen varias formas de codificar textos. Usaremos la frecuencia de término inversa a la frecuencia en documentos (TF-IDF: *Term Frequency-Inverse Document Frequency*). TF-IDF transforma un texto de palabras en un vector numérico donde cada palabra tiene una puntuación.
- Palabras que aparecen con frecuencia en un texto reciben mayor puntuacion...
- ... pero palabras que aparecen con mucha frecuencia en todos los documentos reciben una penalizacion.
- Como resultado, **las palabras que son muy distintivas en uno o pocos textos en particular reciben mayor puntuacion**.

Comenzaremos cargando la biblioteca que usaremos para la codificación y la clasificación de texto: `scikit-learn`.

In [ ]:
# Para instalar scikit-learn:
#!pip install scikit-learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Ahora creamos el codificador TF-IDF:

In [ ]:
# Importante: el input del codificador TF-IDF de scikit-learn es una lista con los textos
corpus = list(df['nombre_item_final'].apply(lambda x: ' '.join(x)))

In [ ]:
corpus

In [ ]:
# Inicializando el codificador
codificador = TfidfVectorizer(stop_words = ['ref'], max_features=500)

# Codificando
vectores = codificador.fit_transform(corpus)

- el argumento `stop_words` nos permite anadir palabras que deben ser ignoradas por el codificador.
    - Esto puede utilizarse si palabras sin significado fueron omitidas en la preparacion de datos
- `max_features` indica cuanto es el maximo de palabras en el corpus que vamos a codificar. Recuerda que nuestro corpus tiene 2,000+ palabras, pero muchas de ellas solo se repiten una vez en los textos

In [ ]:
len(conteo_total)

In [ ]:
vectores.shape

For an easier understanding of the text encoding, we'll transform this back into a dataframe:

El objeto resultante `vectores` contiene las codificaciones de los 1,000 textos. Cada uno de ellos es un vector con la codificación TF-IDF de las 500 palabras más utilizadas en todo el corpus. Elegir 500 es una decisión arbitraria; recuerda que inicialmente teníamos un total de más de 2,000 palabras. De ahora en adelante, nos referiremos a estas 500 palabras como nuestro **diccionario**.

Para entender mejor la codificación del corpus texto, transformaremos esto de nuevo en un dataframe:

In [ ]:
# Columnas para el dataframe
diccionario = codificador.get_feature_names_out()
diccionario[:70]

In [ ]:
# Contenido del dataframe
vectores_data = vectores.todense()

In [ ]:
df_tfidf = pd.DataFrame(data=vectores_data, columns=diccionario)

In [ ]:
df_tfidf

Dos puntos importantes sobre este resultado:

- La matriz `tf_idf` contiene la misma información que `vectores`, excepto que está transformada en un dataframe de Pandas con nombres de columnas e IDs de los artículos.
    - Este paso no era realmente necesario pero lo añadimos para entender mejor el resultado de la codificación
    - La mayoría de ejemplos de analisis de textos omitirán este paso y trabajarán directamente con `vectores`.
- El resultado en `df_tfidf` y `vectores` es una matriz con **muchísimos** ceros.
    - Esto sucede porque la codificación asigna un puntaje de cero a las palabras que están en el diccionario pero que no aparecen en ese documento. Este es un resultado esperable en la codificación TF-IDF.

La información real en estos datos es escasa y se dispersa a lo largo de las muchas dimensiones (columnas) de los datos. Vamos a reducir la dimensionalidad de los datos con análisis de componentes principales (PCA) a solo dos dimensiones. Esto también nos permitirá visualizar la proximidad entre textos.

## 2.2 Análisis de componentes principales (*PCA - Principal Component Analysis*)

In [ ]:
import sklearn.decomposition
PCA = sklearn.decomposition.PCA

In [ ]:
pca = PCA(n_components = 2).fit(vectores) # inicializando y ajustando el modelo PCA
vectores_reducidos = pca.transform(vectores)    # transformando los datos

In [ ]:
vectores_reducidos.shape

El resultado ahora tiene solo dos valores por cada texto. Podemos visualizar los valores para entender mejor que paso aqui. En resumen, transformamos la matriz `df_tfidf` en esto:

In [ ]:
pd.DataFrame(vectores_reducidos)

Podemos visualizar estos dos componentes resultantes. Los valores específicos no tienen ningun significado en concreto, pero la **proximidad entre los valores** indica que esos tenian tenían una codificación TF-IDF similar, lo que significa que están "cerca" en cuanto a las palabras que contienen.

Además, vamos a usar en la visualizacion la variable `clase` para ver que tanto se asemejan textos pertenecientes a una misma clase.

In [ ]:
# Figura
#fig = plt.figure(figsize = (10,6))
plot = sns.scatterplot(
    x = vectores_reducidos[:, 0], 
    y = vectores_reducidos[:, 1], 
    hue = df['clase'],
    s = 10
)

# Personalizacion
plt.legend(title='Clase')
sns.move_legend(plot, "upper left", bbox_to_anchor=(1, 1))
plt.xticks(())
plt.yticks(())
plt.axis('off')
plt.show()

## 2.3 Clasificacion supervisada

Despues del resultado del PCA, podemos continuar con construir nuestros cladsificadores.

Antes de seguir, pausemos un momento para repasar todos los pasos que hemos seguido para la clasificación de texto hasta este punto:

1. Comenzamos con los textos en bruto e hicimos la preparación del texto:
    - convertimos los textos a minúsculas
    - eliminamos las palabras vacías (stop words) y los números
    - lematizamos las palabras
2. Luego codificamos los datos preparados usando TF-IDF
3. Después, reducimos las dimensiones de 1000 a 2 y visualizamos el resultado

Vamos a continuar usando los resultados del paso (3) para entrenar nuestros clasificadores supervisado y no supervisado.

### Entrenamiento de un clasificador para clasificación supervisada

Para asignar observaciones a grupos etiquetados, necesitamos hacer una **clasificación supervisada**. En este ejemplo usaremos un **clasificador de *random forest*** (bosque aleatorio), pero ten en cuenta que la biblioteca que estamos utilizando (**scikit-learn**) ofrece otros tipos de clasificadores tambien.

- [Clasificadores de metodos de agrupacion de datos (*ensemble*)](https://scikit-learn.org/stable/api/sklearn.ensemble.html)
- [Clasificadores de metodos bayesianos](https://scikit-learn.org/stable/api/sklearn.naive_bayes.html)
- [Clasificadores de metodos de vectores de soporte (*support vector machine - SVM*)](https://scikit-learn.org/stable/api/sklearn.svm.html)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Inputs para el clasificador
x = vectores_reducidos  # datos para la clasificacion
y = df['clase']        # clases que vamos a predecir

In [ ]:
# Inicializando el clasificador
clasificador = RandomForestClassifier(class_weight='balanced')

**Importante:** usar el argumento `class_weight` igual a `'balanced'` es crucial dado que esto ajusta la importancia de predecir clases sub representadas en `y`. Omitir esto da como resultado que el clasificador tienda a producir un mejor resultado para las clases mas representadas.

In [ ]:
clasificador.fit(x, y)

Después de esto, el clasificador ha sido entrenado con los datos en `x` para aprender qué patrones en ellos producen los resultados en `y` (las clases).

### Clasificación

Ahora vamos a clasificar nuestros textos con el clasificador que entrenamos.

In [ ]:
clasificaciones = classifier.predict(x)

In [ ]:
# Visualizando algunas de las clasificaciones
clasificaciones[:10]

In [ ]:
# Tabulando los valores clasificados
pd.Series(clasificaciones).value_counts()

In [ ]:
# agregando las clasificaciones al dataframe
df['clasificacion'] = clasificaciones

In [ ]:
df

In [ ]:
# Revisando las clases reales y clasificaciones
pd.crosstab(df['clase'], df['clasificacion'])

Precision total:

In [ ]:
# Estimando la precision total: (textos clasificados correctamente) / (total)
precision = (df['clase'] == df['clasificacion']).sum() / len(df)
print(f'La precision total es {precision*100}%')

Precision por clases:

In [ ]:
clases = list(df['clase'].unique())
clases

In [ ]:
for clase in clases:
    df_temp = df[df['clase']==clase]
    precision = (df_temp['clase'] == df_temp['clasificacion']).sum() / len(df_temp)
    print(f'La precision de la clase {clase} es {round(precision*100, 1)}%')

Algunas notas sobre este resultado:

- Nuestro clasificador tiene una precision total de 77%
    - Este es un buen resultado, mas aun considerando que logramos esto con una muestra relativamente pequena y sin un proceso demasiado complicado
- El clasificador predice mejor unas clases que otras. Esto suele pasar y depende de con que datos se ha entrenado el clasificador. En nuestro caso, los datos son valores numericos derivados de palabras mediante TF-IDF y PCA, asi que clases con palabras mas distintivas seguramente tendran una mejor clasificacion
- Estamos evaluando el desempeno del clasificador con los mismos datos con los que lo entrenamos. Nota que hacemos esto solo por conveniencia, en realidad lo ideal seria dividir los datos totales de forma aleatoria entre una submuestra que usamos para entrenar el clasificador y otra submuestra que usamos para evaluar su desempeno --esto normalmente se conoce como los *train set* y *test set*. Por ejemplo, la division puede ser entre usar el 80% de los datos para el train set y el restanto 20% para el test set.
    - Omitir esto puede llevar a producir un **clasificador sobreajustado** (*overfitted*): que funciona muy bien con los datos con los que se le entreno, pero no sirve para hacer generalizaciones en nuevos textos.

## 2.4 Clasificacion no supervisada

La clasificacion no supervisada tiene la diferencia crucial de que los clasificadores no tienen una variable `y` con las clases que deben clasificar. Mas bien lo que hacen es tomar los datos en `x`, determinar que observaciones son "cercanas" en base a esos datos y asignar clases agnosticas de un significado predeterminado.

El input del clasificador aca tambien seran los resultados del PCA en `vectores_reducidos`. Usaremos el método `KMeans()` del módulo `cluster` de la biblioteca `sklearn` para este clasificador.

In [ ]:
import sklearn.cluster

In [ ]:
# Numero de clases
n = 4

# Inicializando el clasificador
km = sklearn.cluster.KMeans(n_clusters=n, init='k-means++')

In [ ]:
km.fit(vectores_reducidos)

In [ ]:
km.labels_

In [ ]:
pd.Series(km.labels_).value_counts()

In [ ]:
pd.crosstab(df['clase'], km.labels_)

Visualizando los resultados:

In [ ]:
# Figura
#fig = plt.figure(figsize = (10,6))
plot = sns.scatterplot(
    x = vectores_reducidos[:, 0], 
    y = vectores_reducidos[:, 1], 
    hue = km.labels_,
    s = 15
)

# Personalizacion
plt.legend(title='Categoria')
sns.move_legend(plot, "upper left", bbox_to_anchor=(1, 1))
plt.xticks(())
plt.yticks(())
plt.axis('off')
plt.show()

Algunos comentarios:

- La clasificacion no supervisada no produce grupos con un significado predefinido, como si ocurre en la clasificacion supervisada.
    - Esto nos impide estimar la precision, ya que no sabemos exactamente que grupo puede ser analogo a cada categoria existente
- El metodo que usamos, k-means, crea los grupos mas proximos de acuerdo a los datos en `x`. Esto se refleja en la visualizacion de arriba, donde hay un patron claro entre los grupos resultantes y los dos ejes (que son los numeros en `x`)

# Notas finales

## Como mejorar un resultado como este

- Mejora la preparacion de datos:
    - Usa un language model de spaCy mas completo y que haga una lematizacion mas precisa
    - Usa mas expresiones regulares para eliminar palabras que contienen codigos
    - Introduce un espacio entre caracteres de letras y digitos en una misma palabray luego aplica la tokenizacion y lematizacion: esto eliminara codigos y letras sueltas que no tienen significado
- Mejora la pre-clasificacion:
    - Usa mas dimensiones en PCA. Prueba a ver como los resultados con 2-3 mas
    - Anade mas columnas en `x`. En este ejemplo usamos solo 2, provenientes de PCA. Puedes anadir columnas con informacion que no utlilizamos del datafrmae `df`, como dummies por cada valor de algunas de las otras columnas
- Mejora la clasificacion:
    - Amplia la muestra a mas de 1,000 observaciones
    - Separa la muestra en un *train set* y un *test set* para descartar que el modelo esta sobre sobreajustado
    - Prueba otros tipos de clasificadores: esta vez utilizamos un modelo de bosques aleatorios, pero quizas otro modelo podria resultar mejor?

## Siguientes pasos para este analisis

- El archivo de Excel que usamos ayer, *muestra_ejercicio_26_5.xlsx*, tiene una pestana con datos sin clasificar.
    - El siguiente paso seria aplicar todo el proceso de limpieza de datos de texto, codificacion TF-IDF y PCA para clasificar esos textos. Nota que la codificacion TF-IDF y el PCA que apliques debe ser **exactamente igual** que la que aplicamos justo antes de entrenar el modelo. Esto significa que no debes usar el metodo `.fit()` para preparar los datos, sino `.transform()`.

## Otras tareas de análisis de texto

Esta fue una introducción a tareas de análisis y mineria de textos. Otras tareas incluyen:

- Reconocimiento de entidades nombradas (*Named Entity Recognition*): detectar menciones de entidades significativas (lugares, nombres de personas, fechas, etc.) en textos
- Espacios vectoriales y *word embeddings*: transformar textos o palabras en vectores de "significados", en lugar de usar codificadores basados en la presencia de palabras como TF-IDF. Esto permite descomponer un texto en sus significados en lugar de en la presencia de palabras, lo cual es un mejor enfoque para muchas tareas de clasificacion **pero requiere mas poder computacional**
- Analisis de sentimientos: consiste en detectar el tono emocional de un texto, usualmente una oracion o parrafo. No es muy relevante para este tipo de datos, pero es una tarea comun

### Large Language Models (LLMs)

No cubrimos los LLMs porque no forman parte de una sesión introductoria. Si te interesa aprender más sobre ellos, puedes revisar recomendamos estas lecturas (en ingles):

- BERT fue el primer (¿o al menos uno de los primeros?) LLM publicado. Este artículo explica bien cómo funciona: [BERT Explained: State of the art language model for NLP](https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270)
- Este es un tutorial sobre cómo trabajar con BERT y ajustarlo (*fine-tune*) para tareas específicas de análisis de texto: [BERT Fine-Tuning Tutorial with PyTorch](https://mccormickml.com/2019/07/22/BERT-fine-tuning/)